In [323]:
import pandas as pd
import os
import pgeocode
import numpy as np

In [347]:
def process_app_records(file_path):

    # Read the file with '|' as the delimiter
    df = pd.read_csv(file_path, delimiter="|", header=None, dtype=str)

    # Filter for 'APP' records
    app_records = df[df[0] == "APP"].reset_index(drop=True)

    # Assign the 52 column names exactly as defined in the document
    app_columns = [
        "Record_Type", "Applicant_FRN", "Auction_ID", "File_Number", "Applicant_Name",
        "First_Name", "Middle_Name", "Last_Name", "Suffix", "State_Or_Citizenship",
        "Applicant_Status", "Legal_Classification", "Reserved1", "Reserved2", "Reserved3",
        "Noncommercial_Status", "Address1", "Address2", "City", "State", "Zip_Code",
        "Country", "Bidding_Option", "Bidding_Credit", "New_Entrant_Credit",
        "Gross_Revenue_Lower", "Gross_Revenue_Upper", "Closed_Bidding_Eligibility",
        "Bidding_Credit_Percentage", "Certifier_First_Name", "Certifier_Middle_Initial",
        "Certifier_Last_Name", "Certifier_Suffix", "Certifier_Title", "Prior_Defaulter",
        "Financial_Statement_Type", "Gross_Revenue_Most_Recent", "Recent_Year_End",
        "Gross_Revenue_One_Year_Prior", "One_Year_Prior_End", "Gross_Revenue_Two_Years_Prior",
        "Two_Years_Prior_End", "Total_Assets", "Aggregate_Gross_Revenues",
        "Aggregate_Gross_Revenues_1", "Aggregate_Gross_Revenues_2", "Aggregate_Total_Assets",
        "Aggregate_Total_Assets_1", "Aggregate_Total_Assets_2", "Financial_Statement",
        "Aggregate_Financial_Statement", "Aggregate_Credits"
    ]

    # Assign the column names
    app_records.columns = app_columns

    return app_records

In [348]:
# Folder containing the .txt files
folder_path = "/Users/Enzo/Desktop/JMP/form175_data"

# Process all .txt files in the folder
all_dataframes = []
for file_name in os.listdir(folder_path):
    if file_name.endswith(".txt"):
        file_path = os.path.join(folder_path, file_name)
        df = process_app_records(file_path)
        all_dataframes.append(df)

# Concatenate all dataframes into one unique dataframe
final_dataframe = pd.concat(all_dataframes, ignore_index=True)

In [349]:
### CLEANING

# Remove duplicates
bidder_data_form175 = final_dataframe.drop_duplicates().reset_index(drop=True)

path_to_data = '/Users/Enzo/Desktop/JMP/114402-V1/Replication-Fox-and-Bajari/data/'
bidder_data = pd.read_csv(path_to_data + 'biddercblk_03_28_2004_pln.csv')


# # Extract rows where 'co_name' is not in 'Applicant_Name'
# missing_rows_bidder_data = bidder_data[~bidder_data['co_name'].isin(bidder_data_form175['Applicant_Name'])]
# missing_rows_form_175 = bidder_data_form175[~bidder_data_form175['Applicant_Name'].isin(bidder_data['co_name'])]

bidder_data_form175 = bidder_data_form175.copy()
bidder_data_form175['co_name'] = bidder_data_form175['Applicant_Name']

index_to_update = bidder_data_form175[~bidder_data_form175['Applicant_Name'].isin(bidder_data['co_name'])].index

bidder_data_form175.loc[index_to_update[0], 'co_name'] = 'Betty A. Gleaton'
bidder_data_form175.loc[index_to_update[1], 'co_name'] = 'HARVEY LEONG'
bidder_data_form175.loc[index_to_update[2], 'co_name'] = 'William Ingram'
bidder_data_form175.loc[index_to_update[3], 'co_name'] = 'ELIZABETH R. GUEST'
bidder_data_form175.loc[index_to_update[4], 'co_name'] = 'Mark M. Guest'
bidder_data_form175.loc[index_to_update[5], 'co_name'] = 'Vincent  D. McBride'
bidder_data_form175.loc[index_to_update[6], 'co_name'] = 'ADILIA M. AGUILAR'
bidder_data_form175.loc[index_to_update[7], 'co_name'] = 'Shawn Capistrano'
bidder_data_form175.loc[index_to_update[8], 'co_name'] = 'GLENN ISHIHARA'
bidder_data_form175.loc[index_to_update[9], 'co_name'] = 'Harold L. Sudbury, Jr.'

Get county from zipcode of each bidder address.

In [351]:
# Get the county and state for each zip code
def get_county_and_state(zip_code):

    nomi = pgeocode.Nominatim('us')  
    location = nomi.query_postal_code(str(zip_code))

    if location.county_name is np.nan or location.state_code is np.nan:
        print('Missing data for zip code:', zip_code)
        return None, None
    
    return location.county_name, location.state_code

# Add a column Zip_Code_cleaned with the first 5 characters of the Zip_Code column
bidder_data_form175['Zip_Code_cleaned'] = bidder_data_form175['Zip_Code'].str[:5]

# Apply the function to the Zip_Code_cleaned column
bidder_data_form175['County'], bidder_data_form175['State_from_zipcode'] = zip(*bidder_data_form175['Zip_Code_cleaned'].apply(get_county_and_state))

Missing data for zip code: 96910
Missing data for zip code: 96911
Missing data for zip code: 00715
Missing data for zip code: 00901
Missing data for zip code: 96910


Obtain bta for each bidder address.

In [353]:
# Drop columns which have all nan values
bidder_data_form175 = bidder_data_form175.dropna(axis=1, how='all')

relevant_columns = ['co_name', 'Address1', 'Address2', 'City', 'State','County' ,'Zip_Code_cleaned' , 'Zip_Code','Applicant_Status', 
                    'Legal_Classification', 'Certifier_Middle_Initial', 'Certifier_Last_Name','Certifier_Suffix']

bidder_data_form175 = bidder_data_form175[relevant_columns]


# Add a column to bidder_data_form175 with the corresponding BTA from county_bta_mapping_data
county_bta_mapping_data = pd.read_csv('/Users/Enzo/Desktop/JMP/FCC_bta_data/btacnty1990.txt', encoding='latin1')

In [355]:
bidder_data_form175.loc[bidder_data_form175['County'].isnull()] 

,co_name,Address1,Address2,City,State,County,Zip_Code_cleaned,Zip_Code,Applicant_Status,Legal_Classification,Certifier_Middle_Initial,Certifier_Last_Name,Certifier_Suffix
30,"Western Systems, Inc.",530 W. O'Brien Drive,NaN,Agana,GU,None,96910,96910,NaN,Corporation,Lee,NaN,Holmes
31,"IT&E Overseas, Inc.",1010 South Marine Drive,NaN,Tamuning,GU,None,96911,96911,Minority Owned Business,Corporation,John,M,Borlas
232,"Telecell Systems, Inc.",X-1 Vayas Torres,NaN,Mercedita,PR,None,00715,00715,Minority Owned Business,Corporation,Hector,R,Gonzalez
233,"PCS 2000, L.P.",361 SAN FRANCISCO STREET,FIRST FLOOR,SAN JUAN,PR,None,00901,00901,Minority Owned Business,Limited Partnership,JAVIER,O,LAMOSO
234,"TELEPACIFIC NETWORK, INC.",259 Martyr St.,Ste. 101,Agana,GU,None,96910,96910,Minority Owned Business,Corporation,Jose,S,Perez


In [364]:
# # Edit counties by hand to match the names in county_bta_mapping_data
bidder_data_form175.loc[bidder_data_form175['County'] == 'City and County of San Francisco', 'County'] = 'San Francisco'
bidder_data_form175.loc[bidder_data_form175['County'] == 'Anchorage Municipality', 'County'] = 'Anchorage'
bidder_data_form175.loc[bidder_data_form175['County'] == 'St. Louis (city)', 'County'] = 'St. Louis'
bidder_data_form175.loc[bidder_data_form175['County'] == 'City of Alexandria', 'County'] = 'Alexandria City'
bidder_data_form175.loc[bidder_data_form175['County'] == 'East Baton Rouge Parish', 'County'] = 'East Baton Rouge'
bidder_data_form175.loc[bidder_data_form175['County'] == 'Waynesboro (city)', 'County'] = 'Waynesboro City'
bidder_data_form175.loc[bidder_data_form175['County'] == 'Jefferson Parish', 'County'] = 'Jefferson'
bidder_data_form175.loc[bidder_data_form175['County'] == 'Western Connecticut', 'County'] = 'Fairfield'
bidder_data_form175.loc[bidder_data_form175['County'] == 'Capitol Region', 'County'] = 'Hartford'


# Merge bidder_data_form175 with county_bta_mapping_data on 'County' and 'State'
merged_data = bidder_data_form175.merge(county_bta_mapping_data[['County', 'State', 'BTA']], 
                                         on=['County', 'State'], 
                                         how='left')

# If no match is found, the 'BTA' column will be NaN, and we can assign it directly
bidder_data_form175['bta'] = merged_data['BTA']


# # Add PR and GU by hand
print('BTAs in PR:', county_bta_mapping_data[county_bta_mapping_data['State'] == 'PR']['BTA'].unique())
print('BTAs in GU:', county_bta_mapping_data[county_bta_mapping_data['State'] == 'GU']['BTA'].unique())

bidder_data_form175.loc[bidder_data_form175['State'] == 'GU', 'bta'] = 490
bidder_data_form175.loc[bidder_data_form175['City'] == 'Mercedita', 'bta'] = 489
bidder_data_form175.loc[bidder_data_form175['City'] == 'SAN JUAN', 'bta'] = 488

# Make column bta integer
bidder_data_form175['bta'] = bidder_data_form175['bta'].astype(int)

BTAs in PR: [489 488]
BTAs in GU: [490]


In [372]:
bidder_data_form175.loc[bidder_data_form175['bta'].isnull()]

,co_name,Address1,Address2,City,State,County,Zip_Code_cleaned,Zip_Code,Applicant_Status,Legal_Classification,Certifier_Middle_Initial,Certifier_Last_Name,Certifier_Suffix,bta


In [371]:
bidder_data_form175['bta']

0       41
1      171
2      381
3       77
4      375
      ... 
250    321
251      7
252    352
253    447
254    430
Name: bta, Length: 255, dtype: int64

In [373]:
# Define the file path
output_file = os.path.join(folder_path, "bidder_data_form175.csv")

# Check if the file exists
if not os.path.exists(output_file):
    # Save the DataFrame to a .csv file
    bidder_data_form175.to_csv(output_file, index=False)
    print(f"File saved at: {output_file}")
else:
    print(f"File already exists: {output_file}")


File saved at: /Users/Enzo/Desktop/JMP/form175_data/bidder_data_form175.csv
